<a href="https://colab.research.google.com/github/mikael-daniels/Datasets/blob/master/RNN_for_NLP_and_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, GlobalMaxPooling1D, Dense

In [3]:
!wget https://lazyprogrammer.me/course_files/spam.csv

--2020-09-17 17:20:53--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.31.81.48, 104.31.80.48, 172.67.213.166, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.31.81.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv.1’

spam.csv.1          100%[===================>] 491.86K  --.-KB/s    in 0.05s   

2020-09-17 17:20:54 (10.6 MB/s) - ‘spam.csv.1’ saved [503663/503663]



In [4]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
new_df = df.iloc[:, :-3]
new_df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
new_df.columns = ['labels', 'data']
new_df.head()

,labels,data
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
new_df['b_labels'] = new_df['labels'].map({'ham':0, 'spam':1})
new_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,labels,data,b_labels
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
target = new_df['b_labels'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(new_df['data'], target, test_size=0.33)

In [10]:
MAX_VOCAB = 20000
tokenizer = Tokenizer(num_words=MAX_VOCAB)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [11]:
word2idx = tokenizer.word_index
V = len(word2idx)
V

7219

In [12]:
X_train_pad = pad_sequences(X_train)
MAX = X_train_pad.shape[1]
MAX

189

In [13]:
X_test_pad = pad_sequences(X_test, maxlen=MAX)
X_test_pad.shape

(1839, 189)

In [18]:
model = Sequential()

D = 20
M = 15

model.add(Embedding(V+1, D, input_length=MAX))
model.add(LSTM(M, return_sequences=True))

model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(X_train_pad, y_train, epochs=10, validation_data=(X_test_pad, y_test))

Epoch 1/10
117/117 [==============================] - 8s 71ms/step - loss: 0.4674 - accuracy: 0.8620 - val_loss: 0.3902 - val_accuracy: 0.8624
Epoch 2/10
117/117 [==============================] - 8s 67ms/step - loss: 0.3228 - accuracy: 0.8677 - val_loss: 0.2914 - val_accuracy: 0.8624
Epoch 3/10
117/117 [==============================] - 8s 67ms/step - loss: 0.2543 - accuracy: 0.8677 - val_loss: 0.2370 - val_accuracy: 0.8624
Epoch 4/10
117/117 [==============================] - 8s 67ms/step - loss: 0.2011 - accuracy: 0.8749 - val_loss: 0.1827 - val_accuracy: 0.9478
Epoch 5/10
117/117 [==============================] - 8s 67ms/step - loss: 0.1327 - accuracy: 0.9826 - val_loss: 0.1091 - val_accuracy: 0.9821
Epoch 6/10
117/117 [==============================] - 8s 67ms/step - loss: 0.0733 - accuracy: 0.9906 - val_loss: 0.0727 - val_accuracy: 0.9875
Epoch 7/10
117/117 [==============================] - 8s 67ms/step - loss: 0.0445 - accuracy: 0.9946 - val_loss: 0.0546 - val_accuracy: 0.9891

In [20]:
model.evaluate(X_test_pad, y_test)

58/58 [==============================] - 1s 12ms/step - loss: 0.0410 - accuracy: 0.9913


[0.040969185531139374, 0.9912996292114258]